### Chroma DB (LangChain 또는 Native API 기준) 으로 필요 기능에 따라 하이브리드 방식으로 구현 필요

| 기능                   | LangChain (Chroma)       | Native Chroma | 비고                       |
| -------------------- | ------------------------ | ------------- | ------------------------ |
| ✅ 벡터 유사도 검색          | O                        | O             | 둘 다 가능                   |
| ✅ RAG 파이프라인 통합       | O                        | ❌             | LangChain 전용             |
| ❌ 전체 문서 조회           | X                        | O             | `collection.get()`으로 가능  |
| ❌ 문서 개수 확인           | X                        | O             | `len(get(...))`          |
| ❌ 조건 필터만 조회          | X                        | O             | `where=` 지원              |
| ❌ 컬렉션 삭제             | X                        | O             | `delete_collection()` 지원 |
| ✅ 외부 embedding 모델 사용 | O (embedding\_function=) | O             | SentenceTransformer 등    |
| ✅ 멀티 컬렉션 관리          | ⚠️ 불편                    | O             | Native 쪽이 유리             |


 - 권장 아키텍처 -
💾 ./chroma_db/

├── manual_user_collection (데이터 저장)

└── 기타 컬렉션

🧠 LangChain: RAG + Retriever 구성

📡 Native Chroma: 관리, 쿼리, 삭제, 필터링


 ### 1. 데이터 적재 및 유지관리 (📦 Native Chroma)

In [31]:
# 1. 데이터 적재 (Native Chroma)
import chromadb
import json
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

from dotenv import load_dotenv
import os

# 임베딩 함수 정의
embedding_fn = OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),  # 환경변수에서 API 키 읽기
    model_name="text-embedding-3-small"
)

client = chromadb.PersistentClient(path="./chroma_db")

collection = client.get_or_create_collection(
    name="manual_user_collection",
    embedding_function=embedding_fn
)

# 예시 데이터 로드
with open("../manual/user/firstUser/approval/approval.json", "r", encoding="utf-8") as f:
    data = json.load(f)

for i, item in enumerate(data):
    collection.add(
        documents=[item["section"]],
        metadatas=[{
            "manual_type": item["manual_type"],
            "user_type": item["user_type"],
            "source_url": item["source_url"],
            "image_urls": json.dumps(item["image_urls"])
        }],
        ids=[f"doc_{i}"]
    )

### ✅ 2-1. 사용자 질문 응답 (RAG: LangChain + Chroma + Retriever) --> similarity_search / 반환: List[Document]

In [22]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 동일한 경로로 연동
vectorstore = Chroma(
    collection_name="manual_user_collection",
    persist_directory="./chroma_db",
    embedding_function=embedding
)

# RAG 기반 검색
query = "네임스페이스 승인 방법"
results = vectorstore.similarity_search(query, k=3)

for doc in results:
    print(doc.page_content)
    print(doc.metadata)
    print("-" * 30)


신청 결재1. 승인 승인버튼 클릭하면 확인 팝업 호출됩니다. 확인버튼 클릭하면 다음단계로 진행됩니다. 1단계에서 승인  2단계로 진행 2단계에서 승인  결재완료, 네임스페이스 생성 --- 2. 반려 반려버튼 클릭하면 확인 팝업 호출됩니다. 반려사유 입력 후, 확인버튼 클릭하면 해당 신청은 반려됩니다.
{'image_urls': '["https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/approval_approve.png", "https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/approval_reject.png"]', 'manual_type': 'firstUser', 'source_url': 'https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/approval', 'user_type': 'firstUser'}
------------------------------
신청 목록 운영자 유저의 목록 모든 신청 내역이 표시 --- 사용자 유저의 목록 내가 신청한 내역만 표시 신청한 네임스페이스 목록이 표시됩니다. 진행상태 종류는 아래와 같습니다. 결재 1단계  운영자가 결재하는 단계 결재 2단계  프로젝트 관리자가 결재하는 단계 결재 완료  결재가 완료되어 네임스페이스 생성 반려  반려되어 네임스페이스 신청 거절된 상태 신청내역 목록이라 Namespace 메뉴의 목록과 다를 수 있습니다. 각 필드에 단어가 포함된 목록은 검색됩니다. 아래는 검색 예시 입니다. ---
{'image_urls': '["https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/approvalList.png", "https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/approvalList_

### ✅ 2-2. 사용자 질문 응답 (RAG: LangChain + Chroma + Retriever) --> similarity_search_with_score / 반환: List[Tuple[Document, float]]

| Distance 값 | 의미     |
| ---------- | ------ |
| 0.0 \~ 0.1 | 매우 유사  |
| 0.1 \~ 0.3 | 유사     |
| 0.3 \~ 0.6 | 약간 관련  |
| 0.6 이상     | 유사도 낮음 |

✅ similarity = 1 - distance로 변환하면 직관적인 스코어처럼 활용 가능

In [27]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 동일한 경로로 연동
vectorstore = Chroma(
    collection_name="manual_user_collection",
    persist_directory="./chroma_db",
    embedding_function=embedding
)

query = "네임스페이스 결재 승인 방법"
results = vectorstore.similarity_search_with_score(query, k=3)

for i, (doc, score) in enumerate(results):
    print(f"[{i+1}] 유사도 스코어: {score:.4f}")
    print(f"문서 내용: {doc.page_content[:80]}...")
    print(f"메타데이터: {doc.metadata}")
    print("-" * 40)

### ✅ 3. 통계 / 모니터링 / 백업 (📊 Native)

In [ ]:
# 전체 문서 개수
# include 파라미터는 IncludeEnum 타입만 허용
# documents, embeddings, metadatas, distances, uris, data 중 선택 필요
all_docs = collection.get()             # 기본적으로 ids 포함됨
print(f"전체 문서 수: {len(all_docs)}개")

# 메타데이터 통계 집계 예: manual_type 별 count
from collections import Counter

metadatas = collection.get(include=["metadatas"])["metadatas"]
manual_types = [meta["manual_type"] for meta in metadatas]
print(Counter(manual_types))

전체 문서 수: 7개
Counter({'firstUser': 4})


### ✅ 4. 조건 필터링 쿼리 (🔍 Native + where)

In [33]:
# 이미지가 존재하는 문서만 조회
# where 조건으로 image_urls 필드가 빈 배열이 아닌 문서 필터링
# include로 documents와 metadatas 데이터만 가져옴
filtered = collection.get(
    where={"image_urls": {"$ne": json.dumps([])}},  # 빈 배열이 아닌 조건
    include=["documents", "metadatas"]  # 필요한 필드만 조회
)

# 필터링된 문서와 메타데이터를 순회하며 출력
for doc, meta in zip(filtered["documents"], filtered["metadatas"]):
    print(f"문서 내용 (앞 100자): {doc[:100]}")  # 문서 내용 일부 출력
    print(f"메타데이터: {meta}")  # 메타데이터 전체 출력
    print("=" * 40)  # 구분선 출력


문서 내용 (앞 100자): 신청 목록 운영자 유저의 목록 모든 신청 내역이 표시 --- 사용자 유저의 목록 내가 신청한 내역만 표시 신청한 네임스페이스 목록이 표시됩니다. 진행상태 종류는 아래와 같습니다. 
메타데이터: {'image_urls': '["https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/approvalList.png", "https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/approvalList_ex.png", "https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/list_search_1.png", "https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/list_search_2.png"]', 'manual_type': 'firstUser', 'source_url': 'https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/approval', 'user_type': 'firstUser'}
문서 내용 (앞 100자): 신청 상세정보 선택한 신청목록의 상세정보를 표시합니다. 신청자, project, cluster, CPU, Memory, Storage 자원 내역 등을 확인할 수 있습니다. 클러스터
메타데이터: {'image_urls': '["https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/img/detail.png"]', 'manual_type': 'firstUser', 'source_url': 'https://doc.tg-cloud.co.kr/manual/console/firstUser/approval/approval', 'user_type': 'firstUser'}
문서 내용 (앞

### ✅ 5. 문서 삭제 / 컬렉션 초기화 (🗑 Native)

In [26]:
# 문서 삭제
collection.delete(ids=["doc_0", "doc_1"])

# 컬렉션 전체 삭제
client.delete_collection("manual_user_collection")


### ✅ 전체 아키텍처 흐름 요약

📁 chroma_db/

└── manual_user_collection/

📌 데이터 적재:       chromadb.PersistentClient.add()

📌 유저 질의 응답:    LangChain Chroma.similarity_search()

📌 필터 조건 쿼리:    collection.get(where=...)

📌 통계/모니터링:     collection.get(include=["metadatas"])

📌 삭제/초기화:       delete(ids), delete_collection()
